In [3]:
! pip install pypdf

   ---------------------------------------- 0.0/304.2 kB ? eta -:--:--
   -------------------- ------------------- 153.6/304.2 kB 3.1 MB/s eta 0:00:01
   --------------------------------- ------ 256.0/304.2 kB 3.1 MB/s eta 0:00:01
   ---------------------------------------  297.0/304.2 kB 2.6 MB/s eta 0:00:01
   ---------------------------------------- 304.2/304.2 kB 2.4 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.1.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
! python.exe -m pip install --upgrade pip

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
   -- ------------------------------------- 0.1/1.8 MB 2.4 MB/s eta 0:00:01
   ---- ----------------------------------- 0.2/1.8 MB 3.5 MB/s eta 0:00:01
   ---------- ----------------------------- 0.5/1.8 MB 3.5 MB/s eta 0:00:01
   ------------------ --------------------- 0.9/1.8 MB 4.5 MB/s eta 0:00:01
   ------------------------ --------------- 1.1/1.8 MB 4.7 MB/s eta 0:00:01
   ------------------------------- -------- 1.4/1.8 MB 5.4 MB/s eta 0:00:01
   ---------------------------------- ----- 1.6/1.8 MB 5.0 MB/s eta 0:00:01
   ---------------------------------------  1.8/1.8 MB 5.3 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 4.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [5]:
import pandas as pd
import numpy as np
import os

os.getcwd()

'C:\\Users\\charl\\JBPP'

In [9]:
pdf_path = 'PDF experimentation/pdf_experiment_test.pdf'
digi_metadata = pd.read_csv('PDF experimentation/unit-60349.csv')
digi_metadata

,pid,title,description,location,iiif
0,tsm:3381073,"From Julian Bond to Susan Phillis, 2 Jan 1974",1,NaN,https://iiif.lib.virginia.edu/iiif/tsm:3381073...
1,tsm:3381074,"To Julian Bond from Susan Phillis, 11 Dec 1973",1,NaN,https://iiif.lib.virginia.edu/iiif/tsm:3381074...
2,tsm:3381075,"From Julian Bond to John Britton, 2 Jan 1974",1,NaN,https://iiif.lib.virginia.edu/iiif/tsm:3381075...
3,tsm:3381076,"From Julian Bond to George Howe, 2 Jan 1974",1,NaN,https://iiif.lib.virginia.edu/iiif/tsm:3381076...
4,tsm:3381077,"To Julian Bond from George Howe, 19 Dec 1973",1,NaN,https://iiif.lib.virginia.edu/iiif/tsm:3381077...
5,tsm:3381078,"From Julian Bond to Lourie Fornise, 2 Jan 1974",1,NaN,https://iiif.lib.virginia.edu/iiif/tsm:3381078...
6,tsm:3381079,"To Julian Bond from Lourie Fornise, 30 Nov 1973",1,NaN,https://iiif.lib.virginia.edu/iiif/tsm:3381079...
7,tsm:3381080,"From Julian Bond to Joe Andrews, 2 Jan 1974",1,NaN,https://iiif.lib.virginia.edu/iiif/tsm:3381080...
8,tsm:3381081,"To Julian Bond from Joe Andrews, 11 Dec 1973",1,NaN,https://iiif.lib.virginia.edu/iiif/tsm:3381081...
9,tsm:3381082,"From Julian Bond to Julian [Price], 2 Jan 1974...",1,NaN,https://iiif.lib.virginia.edu/iiif/tsm:3381082...


In [15]:
# need to manually insert output prefix? no reference to PJB ID at the moment? both problems that ought to be fixed
# os.mkdir('PDF experimentation/test-output')
output_dir = 'PDF experimentation/test-output'
pjb_name = 'mss13347-b17-f3'

In [17]:
def page_lengths_from_titles(titles):
    lengths = []
    current_title = titles[0]
    count = 0

    for title in titles:
        if title == current_title:
            count += 1
        else:
            lengths.append(count)
            current_title = title
            count = 1
    lengths.append(count)  # add the last document's count
    return lengths

In [21]:
titles = digi_metadata['title']

page_lengths = page_lengths_from_titles(titles)

In [35]:
from pypdf import PdfReader, PdfWriter

def split_pdf_by_lengths(pdf_path, page_lengths, output_dir=output_dir, output_prefix=pjb_name):
    """
    Splits a PDF into multiple PDFs based on a list of page lengths.

    Args:
        pdf_path (str): Path to the original PDF file.
        page_lengths (List[int]): List of page counts for each desired split.
        output_prefix (str): Prefix for output files (default: 'split').
    """
    reader = PdfReader(pdf_path)
    pages = len(reader.pages)
    current = 0

    for i, length in enumerate(page_lengths):
        if current + length > pages:
            raise ValueError(f'Not enough pages to fulfill document {i+1} with {length} pages.')
        
        writer = PdfWriter()
        for page_num in range(current, current + length):
            writer.add_page(reader.pages[page_num])

        output_path = f"{output_dir}/{output_prefix}_{i+1}.pdf"
        with open(output_path, "wb") as f_out:
            writer.write(f_out)
        print(f"Saved: {output_path}")

        current += length

    if current < total_pages:
        print(f"Warning: {total_pages - current} page(s) were not used.")

In [37]:
split_pdf_by_lengths(pdf_path, page_lengths)

Saved: PDF experimentation/test-output/mss13347-b17-f3_1.pdf
Saved: PDF experimentation/test-output/mss13347-b17-f3_2.pdf
Saved: PDF experimentation/test-output/mss13347-b17-f3_3.pdf
Saved: PDF experimentation/test-output/mss13347-b17-f3_4.pdf
Saved: PDF experimentation/test-output/mss13347-b17-f3_5.pdf
Saved: PDF experimentation/test-output/mss13347-b17-f3_6.pdf
Saved: PDF experimentation/test-output/mss13347-b17-f3_7.pdf
Saved: PDF experimentation/test-output/mss13347-b17-f3_8.pdf
Saved: PDF experimentation/test-output/mss13347-b17-f3_9.pdf


ValueError: Not enough pages to fulfill document 10 with 1 pages.